In [ ]:
import findspark

In [ ]:
findspark.init('/home/nahle/spark-2.1.0-bin-hadoop2.7')

In [ ]:
import pyspark

In [ ]:

from pyspark.sql import SparkSession


In [ ]:
spark = SparkSession.builder.appName("Data Frames").getOrCreate()

# DataFrames
    

## Creating DataFrames from RDDs and defining schemas

In [ ]:
l = [('Lio', 31, 50.0, 'Foot'), 
     ('Lebron', 33, 38.5, 'Basket'), 
     ('Novak', 31, 15.0, 'Tennis'), 
     ('Kobe', 40, 24.5, 'Basket'),
     ('Cristiano', 34, 30.0, 'Foot'),
     ('Neymar',26 ,36.8, 'Foot')]

In [ ]:
rdd= spark.sparkContext.parallelize(l)

In [ ]:
rdd.take(6)

In [ ]:
df = spark.createDataFrame(rdd)

In [ ]:
df.show()

In [ ]:
df1=spark.createDataFrame(rdd, schema=['Name', 'Age', 'Salary', 'Sport'])

In [ ]:
df1.show()

In [ ]:
df1.describe()

In [ ]:
from pyspark.sql.types import *

In [ ]:
mySchema = StructType([
    StructField("Name", StringType(), True),
    StructField("Age", IntegerType(), True),
    StructField("Salary", DoubleType(), True),
    StructField("Sport", StringType(), True),
])

In [ ]:
df2=spark.createDataFrame(rdd, schema=mySchema)

In [ ]:
df2.show()

In [ ]:
df2.groupBy('Sport').avg().select('sport', 'avg(Salary)').show()

In [ ]:
df2.groupBy('Sport').avg('Salary').orderBy('avg(Salary)', ascending=False).show()

In [ ]:
df2.columns

In [ ]:
for i in range(4):
    print (df2.columns[i])

In [ ]:
df2.dtypes

In [ ]:
df2.filter(df2.Salary>30).show()

## Aggregation with DataFrames

In [ ]:
from pyspark.sql import DataFrame
print (DataFrame.agg.__doc__)

In [ ]:
from pyspark.sql import functions as F

In [ ]:
df2.groupBy('Sport').agg(F.avg('Salary').alias("Avg Salary"),
                         F.min('Salary').alias ('Min Salary'),
                         F.max('Salary').alias ('Max Salary'),
                         ).show()

## Saving DataFrame Output

In [ ]:
df2.write.csv('output/players.csv')

# Real Dataset

## Stock data 2000-2018
Financial data of all stocks listed on NASDAQ on 27-08-2018

https://toolbox.google.com/datasetsearch/search?query=stock%20data&docid=%2BSTASkPOS9mpodvoAAAAAA%3D%3D

https://www.kaggle.com/danieljperez/financial-data-20002018/version/1

In [ ]:
## create a dataframe from a csv file
df1 = spark.read.csv('financial_data_2000_2018_cleaned.csv', inferSchema=True, header=True)

In [ ]:
df1.show()

In [ ]:
df1.printSchema()

In [ ]:
help(df1.drop)

In [ ]:
df = df1.drop('Low', 'Adj Close', '_c0')

In [ ]:
df.show(n=5)

In [ ]:
df.groupBy('stock').avg('open').show()

# Now on date and TimesStamps

In [ ]:
from pyspark.sql.functions import (dayofmonth, dayofyear,  
                                   month, hour, 
                                   weekofyear, year, 
                                   format_number, date_format)

In [ ]:
import datetime
from pyspark.sql.types import DateType
from pyspark.sql.functions import udf

def toDate(timeAsInteger):
    return datetime.datetime.fromtimestamp(timeAsInteger)

toDate=udf(toDate, DateType())


In [ ]:
newDF= df.withColumn('date', toDate(df['timestamp'])).drop('_c0').drop('timestamp')

In [ ]:
newDF.show()

In [ ]:
newDF.count()

In [ ]:
newDF.select(year(newDF.date)).show()

In [ ]:
newDF = newDF.withColumn('Year', year(newDF['date']))

In [ ]:
newDF.show()

In [ ]:
newDF.groupby('Year').avg().show()

In [ ]:
newDF.groupby('Year').avg().select(['Year', 'avg(high)']).show()

In [ ]:
vDF = newDF.groupby('Year').agg(F.min('Volume'), F.max('Volume'), F.avg('Volume')).orderBy('Year')

In [ ]:
vDF.show()

In [ ]:
new= vDF.withColumnRenamed('avg(Volume)', 'Avg Yearly Volume')

In [ ]:
new.select('Year', format_number('Avg Yearly Volume', 2).alias('Avg')).show()

In [ ]:
with_week = newDF.withColumn('week', weekofyear(newDF.date))

In [ ]:
with_week.show()

In [ ]:
with_week.groupBy('week').agg(F.avg('volume'), F.max('volume')).orderBy('avg(volume)', ascending=False).show()